In [126]:
import pandas as pd
import numpy as np

In [127]:
import requests
star_response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

In [128]:
from bs4 import BeautifulSoup
star_soup = BeautifulSoup(response.content, "html.parser")

In [129]:
star_table = soup.find_all("table")[0]
star_table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Starbucks Stores 2023<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [130]:
row_1 = star_table.find_all("tr")[1]
state_tag = row_1.find("th")
state_tag.text

'California'

In [131]:
store_tag = row_1.find("td")
store_tag.text

'3,080'

In [132]:
rows = []

# iterate over all rows in the faculty table
for row in star_table.find_all("tr")[1:]:

    # The information we need is the text between tags.

    state_tag = row.find("th")
    state = state_tag.text

    store_tag = row.find("td")
    store = store_tag.text

    # Append this data.
    rows.append({
        "state": state,
        "starbucks_store_2023": store
    })

In [133]:
bux_df = pd.DataFrame(rows)

In [134]:
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")

In [135]:
dunkin_soup = BeautifulSoup(dunkin_response.content, "html.parser")

In [136]:
dunkin_table = dunkin_soup.find_all("table")[0]
dunkin_table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Dunkin Locations 2024<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [137]:
dunkin_rows = []

# iterate over all rows in the faculty table
for row in dunkin_table.find_all("tr")[1:]:

    # The information we need is the text between tags.

    state_tag = row.find("th")
    state = state_tag.text

    store_tag = row.find("td")
    store = store_tag.text

    # Append this data.
    dunkin_rows.append({
        "state": state,
        "dunkin_store_2023": store
    })

In [138]:
dunkin_df = pd.DataFrame(dunkin_rows)

In [139]:
coffee_df = pd.merge(bux_df, dunkin_df, how="inner", on="state")

In [140]:
pop_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")

In [141]:
pop_soup = BeautifulSoup(pop_response.content, "html.parser")

In [142]:
pop_table = pop_soup.find_all("table")[0]

In [145]:
pop_rows = []

# iterate over all rows in the faculty table
for row in pop_table.find_all("tr")[1:]:

    # Get all the cells (<td>) in the row.
    cells = row.find_all("td")

    # The information we need is the text between tags.

    state_tag = cells[2].find("strong") or cells[2]
    state = state_tag.get_text(strip=True)

    pop_tag = cells[3].find("strong") or cells[3]
    pop = pop_tag.get_text(strip=True)

    # Append this data.
    pop_rows.append({
        "state": state,
        "pop_2020": pop
    })

In [151]:
pop_df = pd.DataFrame(pop_rows)
coffee_df = pd.merge(coffee_df, pop_df, how="inner")

In [196]:
bux_stock = 96.83
dunk_stock = 106.48

coffee_df["starbucks_stock"] = bux_stock
coffee_df["dunkin_stock"] = dunk_stock

In [205]:
state_to_region = {
    "Connecticut": "Northeast", "Maine": "Northeast", "Massachusetts": "Northeast",
    "New Hampshire": "Northeast", "Rhode Island": "Northeast", "Vermont": "Northeast",
    "New Jersey": "Northeast", "New York": "Northeast", "Pennsylvania": "Northeast",
    
    "Illinois": "Midwest", "Indiana": "Midwest", "Michigan": "Midwest", 
    "Ohio": "Midwest", "Wisconsin": "Midwest", "Iowa": "Midwest", 
    "Kansas": "Midwest", "Minnesota": "Midwest", "Missouri": "Midwest", 
    "Nebraska": "Midwest", "North Dakota": "Midwest", "South Dakota": "Midwest",
    
    "Delaware": "South", "Florida": "South", "Georgia": "South",
    "Maryland": "South", "North Carolina": "South", "South Carolina": "South",
    "Virginia": "South", "Washington, D.C.": "South", "West Virginia": "South",
    "Alabama": "South", "Kentucky": "South", "Mississippi": "South", 
    "Tennessee": "South", "Arkansas": "South", "Louisiana": "South", 
    "Oklahoma": "South", "Texas": "South",
    
    "Arizona": "West", "Colorado": "West", "Idaho": "West", 
    "Montana": "West", "Nevada": "West", "New Mexico": "West", 
    "Utah": "West", "Wyoming": "West", "Alaska": "West", 
    "California": "West", "Hawaii": "West", "Oregon": "West", 
    "Washington": "West"
}

coffee_df["region"] = coffee_df["state"].map(state_to_region)

In [220]:
def webscrape_company(url):
    """
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find_all("table")[0]

    rows = []

    for row in table.find_all("tr")[1:]:

        state_tag = row.find("th")
        state = state_tag.text

        store_tag = row.find("td")
        store = store_tag.text

        rows.append({
            "state": state,
            "stores": store
    })

    return pd.DataFrame(rows)

In [223]:
walmart_df = webscrape_company(url = "https://worldpopulationreview.com/state-rankings/walmart-stores-by-state")
walmart_df

,state,stores
0,Texas,517
1,Florida,341
2,California,280
3,North Carolina,192
4,Georgia,189
5,Illinois,161
6,Ohio,146
7,Missouri,137
8,Tennessee,137
9,Pennsylvania,134
